# Estimation de la dérivée spatiale

Ce problème est étudié dans la section 6.2 du rapport.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import torch 
from utils.data_processing import *
from utils.vizualisation import *

import matplotlib.pyplot as plt
import numpy as np
import h5py
from pathlib import Path
import matplotlib.colors as mcolors
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
import pysindy as ps
import kan
import torch
import time
from torch import autograd
from tqdm import tqdm

### Préparation des données 

Lecture, découpage et ajout des cond limites :

In [ ]:
# Visualisation des résultats
plot_metrics(iters, losses_total, sindy_iters_complet, steps_list, num_pb)
plot_results(t, z, y, pred_sindy, train_id, num_pb)
plot_nrmse_distribution(NRMSE, num_pb)
plot_feature_importance(scores, features, num_pb)

In [32]:
### parametres des données ###
n_run = 1 # numero de la run de la DNS
slicedir = 2 # direction 2 == y fixé
sliceid = 7 # slice 7 == (y=0.25)
idtime = 6080 # temps initial, ici t0 = 291.49825
ntime = 174 # nombre de points temporels
nfin = int(idtime+ntime-1) # temps final, ici tfin = 295.0015
nech=1 # pas de l'échantillonage 
nlaps = np.arange (idtime,nfin+1,nech) # les instants à récupérer
champs = ['U', 'W', 'T'] # les champs à récupérer
###############################

slices, xx, zz, tt = recup_slices(n_run, slicedir, sliceid, idtime, nlaps, champs)
print('Données récupérées : ')
print(f"    {len(tt):.0f}"+" instantanés entre "+f"{tt[0]:.3f}"+" et "+f" {tt[-1]:.3f}") 
print("    Nombre de point en x : "+f"{len(xx):.0f}")
print("    Nombre de point en z : "+f"{len(zz):.0f}")
print("\n")

### parametres des découpes ###
x_cut = 320 # prend segment au centre (x=0.50 et y=0.25)
z_cut = 125 # prend en hauteur de 0.00 à 0.10
cond_lim = True 
###############################

X = slices[x_cut,:z_cut].astype(np.float32)
t = tt.astype(np.float32)
x = [xx[x_cut].astype(np.float32)]
z = zzh[:z_cut].astype(np.float32)
if cond_lim : # conditions limites en z=0 : vitesses nulles et temperature paroi chaude (==1)
    cond_vitesses, cond_temperature = np.zeros((1, len(t), 2)), np.ones((1, len(t), 1))
    assemblage_cond = np.concatenate((cond_vitesses, cond_temperature), axis=2)
    X = np.concatenate((assemblage_cond, X), axis=0) # ajoute sur les données
    z = np.concatenate((np.zeros(1), z)) 
X = np.transpose(X, (1, 0, 2)) # mets les dimensions d'en l'ordre

print('Données découpées : ')
print(f"    {len(t):.0f}"+" instantanés entre "+f"{t[0]:.3f}"+" et "+f" {t[-1]:.3f}") 
print("    Nombre de point en x : "+f"{len(x):.0f}")
print("    Nombre de point en z : "+f"{len(z):.0f}")
print("    Dimensions des données : "+f"{np.shape(X)}")


Données récupérées : 
    174 instantanés entre 291.498 et  295.002
    Nombre de point en x : 640
    Nombre de point en z : 832
Données découpées : 
    174 instantanés entre 291.498 et  295.001
    Nombre de point en x : 1
    Nombre de point en z : 126
    Dimensions des données : (174, 126, 3)


Création des datasets pour SINDy et pour KAN : 

In [53]:
### parametres des datasets ###
normalisation = True # normalise les données d'entrée
test_ratio = 0.3 # test set ratio ==30%
device = 'cpu'
###############################

# créer le label y : les dérivées verticales de u et de theta
y = np.gradient(X[:,:,[0,2]], z, axis=1, edge_order=2)

# calcule la derivée temporelle en entrée pour les KANs
Xt = np.gradient(X, t, axis=0, edge_order=2) 

# créer la matrice d'entrée pour les KANs
X_kan = np.concatenate((X, Xt), axis=2)

# normalisation des données KAN, SINDy le fait dans la bibliothèque
if normalisation : 
    X_kan, mean_X_kan, std_X_kan = normalize(X_kan)

# split train/test 
tscv = TimeSeriesSplit(n_splits=2, test_size=int(t.size*test_ratio))
train_id, test_id = list(tscv.split(t))[1] 

data_kan = {}
data_kan['train_input'] = torch.tensor(X_kan[train_id].reshape(len(z)*len(train_id), X_kan.shape[-1]), dtype=torch.float32, device=device)
data_kan['test_input'] = torch.tensor(X_kan[test_id].reshape(len(z)*len(test_id), X_kan.shape[-1]), dtype=torch.float32, device=device)
data_kan['train_label'] = torch.tensor(y[train_id].reshape(len(z)*len(train_id), y.shape[-1]), dtype=torch.float32, device=device)
data_kan['test_label'] = torch.tensor(y[test_id].reshape(len(z)*len(test_id), y.shape[-1]), dtype=torch.float32, device=device)

# split SINDy
data_sindy = {}
data_sindy['train_input'] = X[train_id]
data_sindy['test_input'] = X[test_id]
data_sindy['train_label'] = y[train_id]
data_sindy['test_label'] = y[test_id]

# to split temporal vector use t[train_id], t[test_id]

print('Datasets prêts : ')
print("    Dimensions pour SINDy | input : "+f"{data_sindy['train_input'].shape}"+ "           | label : "+f"{data_sindy['train_label'].shape}")
print("    Dimensions pour KAN   | input : "+f"{data_kan['train_input'].shape}"+ "  | label : "+f"{data_kan['train_label'].shape}")



Datasets prêts : 
    Dimensions pour SINDy | input : (122, 126, 3)           | label : (122, 126, 2)
    Dimensions pour KAN   | input : torch.Size([15372, 6])  | label : torch.Size([15372, 2])


### SINDy

In [87]:
# Constuction des librairies de fonctions
library_functions = [lambda x: x]
library_function_names = [lambda x: x]

pde_lib = ps.PDELibrary(
    library_functions=library_functions,
    function_names=library_function_names, 
    derivative_order=1, 
    spatial_grid=t[train_id], # grille temporelle
    include_bias=True,
    include_interaction=True,
    is_uniform=False,
)

pde_lib_test = ps.PDELibrary(
    library_functions=library_functions, 
    function_names=library_function_names, 
    derivative_order=1, 
    spatial_grid=t[test_id], # grille temporelle 
    include_bias=True,
    include_interaction=True,
    is_uniform=False,
)

data_sindy['train_transf'] = pde_lib.fit_transform(data_sindy['train_input'])
data_sindy['test_transf'] = pde_lib_test.fit_transform(data_sindy['test_input'])
feature_name = pde_lib.get_feature_names(input_features=champs) 
print('Construction de la librairie de fonctions :')
print("    Nombre de fonctions : "+f"{len(feature_name)}")
print("    Exemples de fonctions : "+f"{[term.replace('_1', '_t') for term in np.random.choice(feature_name, 3, replace=False)]}")

Construction de la librairie de fonctions :
    Nombre de fonctions : 16
    Exemples de fonctions : ['T_t', 'UT_t', 'UW_t']


In [88]:
normalisation

True

In [166]:
# Entrainement avec SR3
n_iters = 100
opt_sindy = ps.SR3(
    threshold=30, 
    thresholder='l1', 
    normalize_columns=normalisation, 
    max_iter=n_iters, 
    verbose=False, 
    tol=1e-5, 
    nu=.5,
)
model_sindy = ps.SINDy(
    feature_library=pde_lib, 
    optimizer=opt_sindy, 
    feature_names=champs,
)

model_sindy.fit(data_sindy['train_input'], x_dot=data_sindy['train_label'], quiet=True)


SINDy(differentiation_method=FiniteDifference(axis=-2),
      feature_library=<pysindy.feature_library.pde_library.PDELibrary object at 0x7f6f456a69f0>,
      feature_names=['U', 'W', 'T'],
      optimizer=SR3(max_iter=100, normalize_columns=True, nu=0.5, threshold=30,
                    thresholder='l1'))

In [167]:
model_sindy.print()

(U)' = -19.631 1 + 1.159 U + 40.097 T
(W)' = 83.389 1 + 15.020 U + -177.461 T + -77.817 U_1 + 159.720 UT_1 + 9.451 TT_1


In [178]:
metrics_sindy = {}
metrics_sindy['mae_train'], metrics_sindy['mae_test'] = [], [] # mean absolute error
metrics_sindy['mse_train'], metrics_sindy['mse_test'] = [], [] # mean squared error
metrics_sindy['complexity'] = [] # complexity == number of nonzero coefficients
for coef in model_sindy.optimizer.history_ : # calcule l'historique des scores
    metrics_sindy['complexity'].append(np.count_nonzero(coef))
    pred_train = (data_sindy['train_transf'] @ coef.T)
    pred_test = (data_sindy['test_transf'] @ coef.T)
    metrics_sindy['mae_train'].append(np.mean(np.abs(pred_train - data_sindy['train_label'])))
    metrics_sindy['mae_test'].append(np.mean(np.abs(pred_test - data_sindy['test_label'])))
    metrics_sindy['mse_train'].append(np.mean((pred_train - data_sindy['train_label'])**2))
    metrics_sindy['mse_test'].append(np.mean((pred_test - data_sindy['test_label'])**2))

# mean total of squares to compute the r2 score (here mtss for each derivative field)
mtss_train = np.mean((data_sindy['train_label'] - data_sindy['train_label'].mean(axis=(0, 1)))**2, axis=(0, 1))
mtss_test = np.mean((data_sindy['test_label'] - data_sindy['test_label'].mean(axis=(0, 1)))**2, axis=(0, 1))
# r2 score
metrics_sindy['r2_train'] = 1 - metrics_sindy['mse_train'] / mtss_train.mean()
metrics_sindy['r2_test'] = 1 - metrics_sindy['mse_test'] / mtss_test.mean()


In [181]:
np.argmax(metrics_sindy['r2_test'])

1

In [182]:
metrics_sindy['r2_test']

array([0.73136349, 0.73326333, 0.73321334, 0.73317702, 0.73281597,
       0.73245551, 0.73204702, 0.7315891 , 0.7311829 , 0.73068571,
       0.73010625, 0.72944956, 0.72871915, 0.72791775, 0.72704754,
       0.72611039, 0.72510788, 0.72404137, 0.7229121 , 0.72172116,
       0.72042161, 0.71901549, 0.71772724, 0.7163903 , 0.71503702,
       0.71374444, 0.71245296, 0.71113371, 0.70966983, 0.70756185,
       0.70600593, 0.70478996, 0.7035401 , 0.70225815, 0.7008438 ,
       0.69936461, 0.69977396, 0.69973935, 0.69927894, 0.69855906,
       0.69767485, 0.69668237, 0.69561541, 0.69138872, 0.68748651,
       0.68401038, 0.68093842, 0.67823018, 0.6758385 , 0.6737161 ,
       0.67181896, 0.67010785, 0.66854879, 0.66711295, 0.66577617,
       0.66451842, 0.66332323, 0.66217715, 0.66106923, 0.65999059,
       0.65893408, 0.65789392, 0.65686547, 0.655845  , 0.65482953,
       0.6538167 , 0.65280459, 0.65179171, 0.65077684, 0.64975906,
       0.64873761, 0.6477119 , 0.64668149, 0.64559069, 0.64387

In [170]:
print(r"Modèle SINDy-PDE avec SR3:")
print("     Nombre de coefficients d'apprentissage :", model_sindy.coefficients().shape)
print("     Complexité du modèle :", metrics_sindy['complexity'][-1])
print("     R2 score | training : ", metrics_sindy['r2_train'][-1], "| testing : ", metrics_sindy['r2_test'][-1]) # r2 
print("     MSE score | training : ", metrics_sindy['mse_train'][-1], "| testing : ", metrics_sindy['mse_test'][-1]) # r2 
print("     RMSE score | training : ", np.sqrt(metrics_sindy['mse_train'][-1]), "| testing : ", np.sqrt(metrics_sindy['mse_test'][-1])) # r2 
print("     MAE score | training : ", metrics_sindy['mae_train'][-1], "| testing : ", metrics_sindy['mae_test'][-1]) # mse
print('\n\n')
model_sindy.print()

Modèle SINDy-PDE avec SR3:
     Nombre de coefficients d'apprentissage : (2, 16)
     Complexité du modèle : 9
     R2 score | training :  0.6987267714482688 | testing :  0.6124343148943332
     MSE score | training :  68.38945322272896 | testing :  101.94821085796201
     RMSE score | training :  8.269791606971058 | testing :  10.096940668240158
     MAE score | training :  5.16021602991799 | testing :  6.325205716125794



(U)' = -19.631 1 + 1.159 U + 40.097 T
(W)' = 83.389 1 + 15.020 U + -177.461 T + -77.817 U_1 + 159.720 UT_1 + 9.451 TT_1


In [ ]:

sindy_iters_complet = range(1, n_iters+1)
sindy_iters = range(10, n_iters+1, 10)
                   
for i in sindy_iters:
    model_sindy.optimizer.max_iter = i
    model_sindy.fit(X_train, x_dot=y_train, quiet=True)

    # metric train/test
    coef = model_sindy.coefficients()
    train_transf = pde_lib.fit_transform(X_train)
    train_pred = train_transf @ coef.T
    test_transf = pde_lib_test.fit_transform(X_test)
    test_pred = test_transf @ coef.T
    MAE_train.append(np.mean(np.abs(train_pred-y_train)))
    MAE_test.append(np.mean(np.abs(test_pred-y_test)))
    R2_train.append(1 - np.mean((train_pred-y_train)**2) / np.mean((y_train - y_train.mean(axis=(0, 1)))**2)  )
    R2_test.append(1 - np.mean((test_pred-y_test)**2) / np.mean((y_test - y_test.mean(axis=(0, 1)))**2)  )
    MSE_train.append(np.mean((train_pred-y_train)**2))
    MSE_test.append(np.mean((test_pred-y_test)**2))
    compl.append(model_sindy.complexity)

nb_coeff = model_sindy.coefficients().shape

mae_train = opt_sindy.mae_history
mse_train = opt_sindy.mse_history
r2_train = opt_sindy.r2_history
compl_hist = np.count_nonzero(np.array(opt_sindy.history_), axis=(1, 2))[:-1]
sindy_mse_train = mse_train
sindy_mse_test = MSE_test
sindy_mae_train = mae_train
sindy_mae_test = MAE_test
sindy_r2_train = r2_train
sindy_r2_test = R2_test
sindy_compl = compl_hist

print(r"Modèle SINDy-PDE avec SR3:")
print("     Nombre de coefficients d'apprentissage :", nb_coeff)
print("     Complexité du modèle :", compl_hist[-1])
print("     R2 score | training : ", R2_train[-1], "| testing : ", R2_test[-1]) # r2 
print("     MSE score | training : ", MSE_train[-1], "| testing : ", MSE_test[-1]) # r2 
print("     RMSE score | training : ", np.sqrt(MSE_train[-1]), "| testing : ", np.sqrt(MSE_test[-1])) # r2 
print("     MAE score | training : ", MAE_train[-1], "| testing : ", MAE_test[-1]) # mse
print('\n\n')
model_sindy.print()